In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# !pip install tensorflow-gpu

In [ ]:
import cudf as cd
import cupy as cp
import tensorflow as tf
from tensorflow import keras as k
import cuml
import numpy as np
import pandas as pd

In [ ]:
train_img_names = cd.read_csv('../input/state-farm-distracted-driver-detection/driver_imgs_list.csv')

In [ ]:
train_img_names.head()

In [ ]:
train_img_names.shape

In [ ]:
train_img_names.info()

In [ ]:
train_img_names['subject'].unique()

In [ ]:
train_img_names['classname'].value_counts()

In [ ]:
from keras.preprocessing import image 
img = image.load_img('../input/state-farm-distracted-driver-detection/imgs/train/c0/img_100026.jpg', 
        target_size = (128, 128))
img

In [ ]:
img = image.img_to_array(img)
img.shape

In [ ]:
train_img_names.drop('subject', 1, inplace = True)

In [ ]:
dic = {'c0': 0, 'c1': 1, 'c2': 2, 'c3': 3, 'c4': 4, 'c5': 5, 'c6': 6, 'c7': 7, 'c8': 8, 'c9': 9}
classes = [dic[str(i)] for i in train_img_names['classname'].to_arrow()]
train_img_names['classname'] = classes
train_img_names.head()

In [ ]:
x = train_img_names['img'].to_pandas()
y = train_img_names['classname'].to_pandas()

In [ ]:
from sklearn.model_selection import train_test_split as tts
x_train, x_test, y_train, y_test = tts(x, y, test_size = 0.3, random_state = 42)

In [ ]:
x_train.head()

In [ ]:
x_train = cd.DataFrame(x_train.reset_index())
x_test = cd.DataFrame(x_test.reset_index())
y_train = cd.DataFrame(y_train.reset_index())
y_test = cd.DataFrame(y_test.reset_index())

In [ ]:
def image_rgb(image_name, cl):
    path = '../input/state-farm-distracted-driver-detection/imgs/train/' + 'c' + str(cl) + '/' + image_name
    mImage = image.load_img(path,target_size = (64, 64))
    mImage = image.img_to_array(mImage)
    return mImage

In [ ]:
xtrain = [image_rgb(x_train['img'][i], y_train['classname'][i]) for i in range(len(x_train))]
xtest = [image_rgb(x_test['img'][i], y_test['classname'][i]) for i in range(len(x_test))]

In [ ]:
xtrain = np.array(xtrain)
xtest = np.array(xtest)

In [ ]:
xtrain.shape

In [ ]:
xtr = xtrain / 255.0
xte = xtest / 255.0

In [ ]:
model = k.Sequential()
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=2,padding='same', activation='relu', input_shape=(64,64,3)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=2,padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=2,padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=2,padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(500, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10, activation='softmax'))
model.summary()

In [ ]:
y_test = y_test.to_pandas().to_numpy()
y_train = y_train.to_pandas().to_numpy()

In [ ]:
y_train = y_train[:, 1]
y_test = y_test[:, 1]

In [ ]:
y_test

In [ ]:
y_train = k.utils.to_categorical(y_train)

In [ ]:
y_test = k.utils.to_categorical(y_test)

In [ ]:
y_train

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(xtr, y_train,epochs=20, validation_data = (xte, y_test), batch_size=32)